In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Статическая ошибка

Подготовка данных

In [2]:
static_df = pd.read_csv("SavedData/NavData__20231125_14h_13m_40s.txt", sep="  ", engine='python')
cropped_static_df = static_df[["OmegaX[°/s]", "OmegaY[°/s]", "OmegaZ[°/s]", "fX[m/s/s]", "fY[m/s/s]", "fZ[m/s/s]"]]

In [3]:
cols_to_replace = ['OmegaX[°/s]', 'OmegaY[°/s]', 'OmegaZ[°/s]', 'fX[m/s/s]', 'fY[m/s/s]', 'fZ[m/s/s]']
for col in cropped_static_df:
    cropped_static_df[col] = cropped_static_df[col].str.replace(',', '.').astype(float)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Возьмем среднее значение по всем столбцам

In [4]:
static_error = pd.DataFrame(cropped_static_df.mean(), columns=['static_mean_rough']).T

Расчитаем проекции на оси:

In [5]:
new_row = []
new_row.append(static_error['OmegaX[°/s]']['static_mean_rough'] * np.sin(70/180 * np.pi) + static_error['OmegaY[°/s]']['static_mean_rough'] * np.cos(70/180 * np.pi))
new_row.append(static_error['OmegaX[°/s]']['static_mean_rough'] * np.cos(70/180 * np.pi) + static_error['OmegaY[°/s]']['static_mean_rough'] * np.sin(70/180 * np.pi))
new_row.append(static_error['OmegaZ[°/s]']['static_mean_rough'])
new_row.append(static_error['fX[m/s/s]']['static_mean_rough'] * np.sin(70/180 * np.pi) + static_error['fY[m/s/s]']['static_mean_rough'] * np.cos(70/180 * np.pi))
new_row.append(static_error['fX[m/s/s]']['static_mean_rough'] * np.cos(70/180 * np.pi) + static_error['fY[m/s/s]']['static_mean_rough'] * np.sin(70/180 * np.pi))
new_row.append(static_error['fZ[m/s/s]']['static_mean_rough'])

Рассмотрим истинные значения

In [6]:
true_values = [15.05 / 3600 * np.sin(55.5/180 * np.pi),
               15.05 / 3600 * np.cos(55.5/180 * np.pi),
               0,
               0,
               0,
               9.8155]

Получим разницу

In [7]:
delta = [abs(new_row[i] - true_values[i]) for i in range(len(new_row))]

In [8]:
static_error = pd.DataFrame(cropped_static_df.mean(), columns=['static_mean_rough']).T

In [9]:
static_error = static_error.append(pd.DataFrame(new_row, columns=['static_mean'], index=cols_to_replace).T)
static_error = static_error.append(pd.DataFrame(true_values, columns=['true_values'], index=cols_to_replace).T)
static_error = static_error.append(pd.DataFrame(delta, columns=['delta_static'], index=cols_to_replace).T)

In [10]:
static_error

,OmegaX[°/s],OmegaY[°/s],OmegaZ[°/s],fX[m/s/s],fY[m/s/s],fZ[m/s/s]
static_mean_rough,-0.259639,-0.406871,-0.482357,-0.326335,0.224587,9.721624
static_mean,-0.383139,-0.471135,-0.482357,-0.229841,0.099430,9.721624
true_values,0.003445,0.002368,0.000000,0.000000,0.000000,9.815500
delta_static,0.386584,0.473503,0.482357,0.229841,0.099430,0.093876


## От запуска, к запуску

In [11]:
datapath = ['SavedData/NavData__20231209_15h_07m_48s_converted.txt',
        'SavedData/NavData__20231209_15h_09m_46s_converted.txt',
        'SavedData/NavData__20231209_15h_11m_06s_converted.txt',
        'SavedData/NavData__20231209_15h_13m_15s_converted.txt',
        'SavedData/NavData__20231209_15h_14m_34s_converted.txt']

cols = ['OmegaX[°/s]', 'OmegaY[°/s]', 'OmegaZ[°/s]', 'fX[m/s/s]', 'fY[m/s/s]', 'fZ[m/s/s]']

run_to_run_error = pd.DataFrame(columns=cols)

Среднее значение по каждому из файлов

In [12]:
for idx in range(len(datapath)):
    run_to_run = pd.read_csv(datapath[idx], sep = "  ", encoding='unicode_escape')[cols]
    mean_row = pd.DataFrame(run_to_run.mean(), columns=[f'mean_file_{idx+1}']).T
    run_to_run_error = run_to_run_error.append(mean_row)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Среднее значение по всем файлам

In [13]:
mean_row = pd.DataFrame(run_to_run_error.mean(), columns=['mean']).T
run_to_run_error = run_to_run_error.append(mean_row)

In [14]:
run_to_run_error = run_to_run_error.append(pd.DataFrame(true_values, columns=['true_values'], index=cols).T)

Вычтем истинные значения

In [15]:
delta_run_to_run = run_to_run_error.diff()
diff_rows = delta_run_to_run.iloc[[6]]
diff_rows = diff_rows.apply(lambda x: abs(x))
diff_rows.rename(index={'true_values':'run_to_run'}, inplace=True)
run_to_run_error = run_to_run_error.append(diff_rows)

In [17]:
run_to_run_error = run_to_run_error.append(static_error.iloc[[3]])
delta_run_to_run = run_to_run_error.diff()
diff_rows = delta_run_to_run.iloc[[8]]
diff_rows = diff_rows.apply(lambda x: abs(x))
diff_rows.rename(index={'delta_static':'delta_run_to_run'}, inplace=True)
run_to_run_error = run_to_run_error.append(diff_rows)

In [18]:
run_to_run_error

,OmegaX[°/s],OmegaY[°/s],OmegaZ[°/s],fX[m/s/s],fY[m/s/s],fZ[m/s/s]
mean_file_1,-0.224272,-0.364186,-0.558750,0.092600,0.009795,9.730271
mean_file_2,-0.225918,-0.330289,-0.537751,0.100827,-0.016662,9.730885
mean_file_3,-0.245540,-0.330819,-0.530966,0.102854,-0.029234,9.734250
mean_file_4,-0.231802,-0.331570,-0.533101,0.098835,-0.023822,9.742209
mean_file_5,-0.229336,-0.353223,-0.558740,0.099922,-0.025765,9.729538
mean,-0.231374,-0.342017,-0.543861,0.099008,-0.017138,9.733431
true_values,0.003445,0.002368,0.000000,0.000000,0.000000,9.815500
run_to_run,0.234819,0.344385,0.543861,0.099008,0.017138,0.082069
delta_static,0.386584,0.473503,0.482357,0.229841,0.099430,0.093876
delta_run_to_run,0.151765,0.129118,0.061504,0.130834,0.082292,0.011806


## В течение работы

In [20]:
in_run_error = pd.DataFrame(columns=cols)
in_run = pd.read_csv('SavedData/NavData__20231209_15h_00m_51s_converted.txt', sep = "  ", encoding='unicode_escape')[cols]
mean_row = pd.DataFrame(in_run.mean(), columns=['mean']).T
in_run_error = in_run_error.append(mean_row)
in_run_error = in_run_error.append(pd.DataFrame(true_values, columns=['true_values'], index=cols).T)
in_run_error = in_run_error.append(static_error.iloc[[3]])
in_run_error = in_run_error.append(run_to_run_error.iloc[[9]])

In [21]:
in_run_error

,OmegaX[°/s],OmegaY[°/s],OmegaZ[°/s],fX[m/s/s],fY[m/s/s],fZ[m/s/s]
mean,-0.241779,-0.363720,-0.548172,0.081479,0.045494,9.723654
true_values,0.003445,0.002368,0.000000,0.000000,0.000000,9.815500
delta_static,0.386584,0.473503,0.482357,0.229841,0.099430,0.093876
delta_run_to_run,0.151765,0.129118,0.061504,0.130834,0.082292,0.011806
